<a href="https://colab.research.google.com/github/sourcecode369/deep-nlp/blob/master/text%20generation/Text_Generation_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LSTM, GRU, Bidirectional 
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

TensorFlow 2.x selected.


In [0]:
import urllib
urllib.request.urlretrieve('https://raw.githubusercontent.com/sourcecode369/unconventional-neural-networks/master/mnist%20generator%20classifier/classify/input.txt','mnist.txt')

('mnist.txt', <http.client.HTTPMessage at 0x7f448787ce10>)

In [0]:
!ls

mnist.txt  sample_data


In [0]:
text = open('mnist.txt', 'rb').read().decode(encoding='utf-8')
text = text.lower()
print("Length of text {} characters.".format(len(text)))

Length of text 46745565 characters.


In [0]:
print(text[:1500])

[[0000000000000000000000000000]
[0000000000000000000000000000]
[0000000000000000000000000000]
[0000000000000000000000000000]
[0000000000000000000000000000]
[0000000000000000011011100000]
[0000000000011111111111100000]
[0000000011111111110000000000]
[0000000011111111110000000000]
[0000000001011100010000000000]
[0000000000011000000000000000]
[0000000000011100000000000000]
[0000000000001100000000000000]
[0000000000000111000000000000]
[0000000000000011100000000000]
[0000000000000001111000000000]
[0000000000000000011100000000]
[0000000000000000011100000000]
[0000000000000001111100000000]
[0000000000000111111100000000]
[0000000000001111110000000000]
[0000000000111111000000000000]
[0000000111111100000000000000]
[0000011111111000000000000000]
[0000111111100000000000000000]
[0000000000000000000000000000]
[0000000000000000000000000000]
[0000000000000000000000000000]]
:[[5555555555]
[5555555555]
[5555555555]
[5555555555]
[5555555555]
[5555555555]
[5555555555]
[5555555555]
[5555555555]
[5555555555

In [0]:
chars = sorted(list(set(text)))
char_to_int = dict((c,i) for i, c in enumerate(chars))
idx_to_char = np.array(chars)
text_as_int = np.array([char_to_int[c] for c in text])

In [0]:
n_chars = len(text)
n_vocab = len(chars)

In [0]:
print("{")
for char, _ in zip(char_to_int, range(20)):
  print(' {:4s}: {:3d}, '.format(repr(char), char_to_int[char]))
print('   ...\n')

{
 '\n':   0, 
 '0' :   1, 
 '1' :   2, 
 '2' :   3, 
 '3' :   4, 
 '4' :   5, 
 '5' :   6, 
 '6' :   7, 
 '7' :   8, 
 '8' :   9, 
 '9' :  10, 
 ':' :  11, 
 '[' :  12, 
 ']' :  13, 
   ...



In [0]:
print(f'{repr(text[:15])} ------ characters mapped to int ------------> {text_as_int[:15]}')

'[[0000000000000' ------ characters mapped to int ------------> [12 12  1  1  1  1  1  1  1  1  1  1  1  1  1]


In [0]:
seq_length = 100

examples_per_epoch = len(text)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
  print(idx_to_char[i.numpy()])

[
[
0
0
0
0
0
0
0
0


In [0]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(''.join(idx_to_char[item.numpy()]))

[[0000000000000000000000000000]
[0000000000000000000000000000]
[0000000000000000000000000000]
[000000
0000000000000000000000]
[0000000000000000000000000000]
[0000000000000000011011100000]
[00000000000111
11111111100000]
[0000000011111111110000000000]
[0000000011111111110000000000]
[0000000001011100010000
000000]
[0000000000011000000000000000]
[0000000000011100000000000000]
[0000000000001100000000000000]

[0000000000000111000000000000]
[0000000000000011100000000000]
[0000000000000001111000000000]
[0000000


In [0]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [0]:
for input_example, target_example in dataset.take(1):
  print('Input data: ', repr(''.join(idx_to_char[input_example.numpy()])))
  print('Target data: ',repr(''.join(idx_to_char[target_example.numpy()])))

Input data:  '[[0000000000000000000000000000]\n[0000000000000000000000000000]\n[0000000000000000000000000000]\n[00000'
Target data:  '[0000000000000000000000000000]\n[0000000000000000000000000000]\n[0000000000000000000000000000]\n[000000'


In [0]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
  print("Step: {:4d}".format(i))
  print(" input: {} ({:s})".format(input_idx, repr(idx_to_char[input_idx])))
  print(" target: {} ({:s})".format(target_idx, repr(idx_to_char[target_idx])))

Step:    0
 input: 12 ('[')
 target: 12 ('[')
Step:    1
 input: 12 ('[')
 target: 1 ('0')
Step:    2
 input: 1 ('0')
 target: 1 ('0')
Step:    3
 input: 1 ('0')
 target: 1 ('0')
Step:    4
 input: 1 ('0')
 target: 1 ('0')


In [0]:
BATCH_SIZE = 512

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset.element_spec

(TensorSpec(shape=(512, 100), dtype=tf.int64, name=None),
 TensorSpec(shape=(512, 100), dtype=tf.int64, name=None))

In [0]:
vocab_size = len(chars)
embedding_dim = 256
rnn_units = 2048

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
                               tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
                               tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
                               tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
                               tf.keras.layers.Dropout(0.2),
                               tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(vocab_size=len(chars), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (512, None, 256)          3584      
_________________________________________________________________
lstm (LSTM)                  (512, None, 2048)         18882560  
_________________________________________________________________
lstm_1 (LSTM)                (512, None, 2048)         33562624  
_________________________________________________________________
dropout (Dropout)            (512, None, 2048)         0         
_________________________________________________________________
dense (Dense)                (512, None, 14)           28686     
Total params: 52,477,454
Trainable params: 52,477,454
Non-trainable params: 0
_________________________________________________________________


In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# batch_size, sequence_length, vocab_size")

(512, 100, 14) # batch_size, sequence_length, vocab_size


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [0]:
print('Input: \n', repr("".join(idx_to_char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n",repr("".join(idx_to_char[sampled_indices])))

Input: 
 '100000000]\n[0000000000000001111100000000]\n[0000000000000011110000000000]\n[00000000000011111000000000'

Next Char Predictions: 
 '097183:\n36691:93[5[7563\n289:329[55\n471[900:0\n97:2260351:8774[]546:2\n:\n\n1]15::83:8]6\n22119:999]4439\n1'


In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Predictions shape: ", example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")
print("Scalar loss: ", example_batch_loss.numpy().mean())

Predictions shape:  (512, 100, 14) # (batch_size, sequence_length, vocab_size)
Scalar loss:  2.6384017


In [0]:
model.compile(optimizer="adam", loss=loss)

In [0]:
import os 

checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS = 2

In [0]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 903 steps
Epoch 1/2
903/903 [==============================] - 1871s 2s/step - loss: 0.3587
Epoch 2/2
903/903 [==============================] - 1871s 2s/step - loss: 0.1867


In [0]:
tf.train.latest_checkpoint(checkpoint_dir=checkpoint_dir)

'./training_checkpoints/ckpt_2'

In [0]:
model = build_model(vocab_size=vocab_size, embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir=checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            3584      
_________________________________________________________________
lstm_2 (LSTM)                (1, None, 2048)           18882560  
_________________________________________________________________
lstm_3 (LSTM)                (1, None, 2048)           33562624  
_________________________________________________________________
dropout_1 (Dropout)          (1, None, 2048)           0         
_________________________________________________________________
dense_1 (Dense)              (1, None, 14)             28686     
Total params: 52,477,454
Trainable params: 52,477,454
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
  num_generate = 10000
  input_eval = [char_to_int[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  temperature = 1.0
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx_to_char[predicted_id])
  return (start_string + ''.join(text_generated))

In [0]:
print(generate_text(model, start_string='['))

[72
2:2[66[66699:2[::2:
:::::22223668:8292[922228:
:
::
878222::[36292::[[:[3:::2[5500000000]1110000000]
[0000000000000000001100000000]
[000000000000000000000]
[0000000000000000000000000000]
[0000000000000000000000000000]
[0000000000000000000000000000]
[0000000000000000000000000000]
[0000000000000000000000000000]
[0000000000000000000111100000]
[0000000000000000000111100000]
[0000000000000000000111000000]
[0000000000000000001110000000]
[0000000000000000000000000000000]
[0000000000000000000000000000]]
:[[3333333333]
[3333333333]
[3333333333]
[3333333333]
[3333333333]
[3333333333]
[3333333333]
[3333333333]
[3333333333]
[3333333333]
[3333333333]
[3333333333]
[3333333333]
[333333333]]:

[[0000000000000000000000000000]
[0000000000000000000000000000]
[0000000]
[0000000000000001100000000000]
[00000000011100000111110000]
[0000000000011100000110000000]
[0000000000000000000100000000]
[0000000000000000001100000000]
[0000000011111000011000000000]
[0000000111111111111000000000]
[00000000111111100000